# Análisis previo a el preprocesamiento

In [95]:
import pandas as pd
import os

## Exploración para los datos Kaggle

In [96]:
# Datos Kaggle
population = pd.read_csv('../data/raw/population.csv', sep=',')
population.head(4)

,year,coo_name,coo,coo_iso,coa_name,coa,coa_iso,refugees,asylum_seekers,returned_refugees,idps,returned_idps,stateless,ooc,oip,hst
0,2010,Afghanistan,AFG,AFG,Afghanistan,AFG,AFG,0,0,0,351907,3366,0,838250,NaN,NaN
1,2010,Iran (Islamic Rep. of),IRN,IRN,Afghanistan,AFG,AFG,30,21,0,0,0,0,0,NaN,NaN
2,2010,Iraq,IRQ,IRQ,Afghanistan,AFG,AFG,6,0,0,0,0,0,0,NaN,NaN
3,2010,Pakistan,PAK,PAK,Afghanistan,AFG,AFG,6398,9,0,0,0,0,0,NaN,NaN


In [97]:
# Valores nulos, tipo de datos y rango de valores
def basic_exploration(df):
    tipo_datos = (df.dtypes).unique()
    for i in tipo_datos:
        print(f'Datos de tipo {i}: ')
        print(df.describe(include=i))
        print('-----------------------------------------------------------------------------------------')
    print('Valores nulos encontrados: ')
    print(df.isnull().sum())

basic_exploration(population)

Datos de tipo int64: 
               year      refugees  asylum_seekers  returned_refugees  \
count  64809.000000  6.480900e+04    64809.000000       64809.000000   
mean    2016.393356  3.440013e+03      564.166119          73.480659   
std        3.722613  5.525597e+04     7455.730894        2460.282939   
min     2010.000000  0.000000e+00        0.000000           0.000000   
25%     2013.000000  5.000000e+00        0.000000           0.000000   
50%     2017.000000  1.200000e+01        8.000000           0.000000   
75%     2020.000000  8.800000e+01       57.000000           0.000000   
max     2022.000000  3.737369e+06   940668.000000      381275.000000   

               idps  returned_idps      stateless           ooc  
count  6.480900e+04   6.480900e+04   64809.000000  6.480900e+04  
mean   7.088696e+03   7.061573e+02     756.625191  4.374958e+02  
std    1.631746e+05   2.365424e+04   19980.220686  2.018268e+04  
min    0.000000e+00   0.000000e+00       0.000000  0.000000e+00  

## Exploración para los datos procedentes del gobierno

In [98]:
filenames = ['Asilo2013.xlsx', 'Asilo2014.xlsx', 'Asilo2015.xlsx', 'Asilo2019.xlsx', 'Asilo2020.xlsx', 'Asilo2021.xlsx']
asilos = {asilo.replace('.xlsx', ''): pd.read_excel(f'../data/raw/{asilo}', sheet_name=None) for asilo in filenames}
longitudes_asilo = {nombre: len(diccionario) for nombre, diccionario in asilos.items()}

print(longitudes_asilo)

{'Asilo2013': 14, 'Asilo2014': 14, 'Asilo2015': 13, 'Asilo2019': 10, 'Asilo2020': 10, 'Asilo2021': 12}


In [99]:
for nombre, asilo in asilos.items():
    print(f'Nombre del archivo: {nombre}')
    for nombre_hoja, df in asilo.items():
        print(f'Nombre de la hoja: {nombre_hoja}')
        print(df.head(5))
        basic_exploration(df)
    print('-----------------------------------------------------------------------------------------\n')

Nombre del archivo: Asilo2013
Nombre de la hoja: Sexo
   Unnamed: 0                                         Unnamed: 1 Unnamed: 2  \
0         NaN  Solicitantes de Protección Internacional por\n...        NaN   
1         NaN                                                NaN        NaN   
2         NaN                                                NaN        NaN   
3         NaN                                                NaN        NaN   
4         NaN                                                           NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  
Datos de tipo float64: 
       Unnamed: 0
count         0.0
mean          NaN
std           NaN
min           NaN
25%           NaN
50%           NaN
75%           NaN
max           NaN
-----------------------------------------------------------------------------------------
Datos de tipo object: 
                

### Exploración para los datos en carpeta procedentes del gobierno

In [100]:
def leer_carpeta(nombre_carpeta):
    ruta_carpeta = f'../data/raw/{nombre_carpeta}'
    lista_carpeta = [archivo for archivo in os.listdir(ruta_carpeta)]
    return lista_carpeta

carpeta12 = leer_carpeta('Asilo2012')
carpeta16 = leer_carpeta('Asilo2016')
carpeta17 = leer_carpeta('Asilo2017')
carpeta18 = leer_carpeta('Asilo2018')

def leer_fichero(filenames, carpeta, ext):
    if ext == '.csv':
        asilos = {asilo.replace(ext, ''): pd.read_csv(f'../data/raw/{carpeta}/{asilo}', sep = ";", encoding='latin-1') for asilo in filenames}
    else:
        asilos = {asilo.replace(ext, ''): pd.read_excel(f'../data/raw/{carpeta}/{asilo}', sheet_name=0) for asilo in filenames}
    return asilos

def mostrar_datos_carpeta(diccionario_asilo):
    for name, df in diccionario_asilo.items():
        print(name)
        print(df)
        basic_exploration(df)
        print('-----------------------------------------------------------------------------------------\n')


asilos2012 = leer_fichero(carpeta12, 'Asilo2012', '.csv')
mostrar_datos_carpeta(asilos2012)

AdmisibilidaddeSolicitudesProtección InternacionalPorpaises
   Nacionalidad   Admitidas  No admitidas  Denegadas  Unnamed: 4
0     AFGANISTAN     45.000             1          0         NaN
1        ALBANIA      5.000             1          0         NaN
2         ANGOLA      6.000             1         10         NaN
3        ARGELIA    171.000             4         25         NaN
4      ARGENTINA      4.000             0          1         NaN
..           ...        ...           ...        ...         ...
92     VENEZUELA     24.000             0          4         NaN
93         YEMEN      5.000             0          0         NaN
94        ZAMBIA      1.000             0          0         NaN
95      ZIMBABWE      4.000             0          0         NaN
96         Total      2.056           112        364         NaN

[97 rows x 5 columns]
Datos de tipo object: 
       Nacionalidad 
count             97
unique            97
top       AFGANISTAN
freq               1
---------

In [101]:
asilos2016 = leer_fichero(carpeta16, 'Asilo2016', '.xlsx')
mostrar_datos_carpeta(asilos2016)

TABLA 2-1
    TABLA 2-1. SOLICITANTES DE PROTECCIÓN INTERNACIONAL POR PAÍS DE ORIGEN Y SEXO  \
0                                       País de origen                              
1                                           Afganistán                              
2                                              Albania                              
3                                             Alemania                              
4                                               Angola                              
..                                                 ...                              
100                                          Venezuela                              
101                                            Vietnam                              
102                                              Yemen                              
103                                           Zimbabwe                              
104                                              Total 

In [102]:
asilos2017 = leer_fichero(carpeta17, 'Asilo2016', '.xlsx')
mostrar_datos_carpeta(asilos2017)

TABLA 2-1
    TABLA 2-1. SOLICITANTES DE PROTECCIÓN INTERNACIONAL POR PAÍS DE ORIGEN Y SEXO  \
0                                       País de origen                              
1                                           Afganistán                              
2                                              Albania                              
3                                             Alemania                              
4                                               Angola                              
..                                                 ...                              
100                                          Venezuela                              
101                                            Vietnam                              
102                                              Yemen                              
103                                           Zimbabwe                              
104                                              Total 

In [103]:
asilos2018 = leer_fichero(carpeta18, 'Asilo2018', '.xlsx')
mostrar_datos_carpeta(asilos2018)

TABLA 2-1
         TABLA 2-1.  \
0    País de origen   
1        Afganistán   
2           Albania   
3            Angola   
4          Apátrida   
..              ...   
119       Venezuela   
120         Vietnam   
121           Yemen   
122        Zimbabwe   
123           Total   

    SOLICITUDES DE PROTECCIÓN INTERNACIONAL PRESENTADAS POR PAÍS DE ORIGEN Y SEXO  \
0                                              Hombres                              
1                                                   78                              
2                                                   42                              
3                                                   10                              
4                                                    4                              
..                                                 ...                              
119                                               9687                              
120                              